# Modeling
In this notebook we perform our Bayesian Logistic Regression using MCMC.

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import statsmodels.api as sm
import networkx as nx
import scipy as sc
import pickle
import pymc3 as pm
import arviz as az
sns.set_theme(context="notebook", font_scale=1.2)
import warnings
warnings.filterwarnings('ignore')

## Data 
Since some of our variables are binary, we perform min-max scaling such that all features are in the range $[0,1]$.

In [2]:
scaler = MinMaxScaler()

In [3]:
with open('feat_cols.pkl', 'rb') as f:
    feat_cols = pickle.load(f)

In [4]:
data = pd.read_csv("ld_clean.csv")
status = data['Status']
data = data[feat_cols + ['Status']]
data = data.infer_objects()
data.fillna(data.mean(), inplace=True)
data[feat_cols] = scaler.fit_transform(data[feat_cols])
data.head()

,co-applicant_credit_type_EXP,Gender_Joint,total_units_3U,Upfront_charges,Neg_ammortization_neg_amm,construction_type_mh,Secured_by_land,Security_Type_Indriect,occupancy_type_sr,Region_North,...,age_<25,approv_in_adv_nopre,age_35-44,lump_sum_payment_lpsm,credit_type_CIB,income,loan_limit_ncf,loan_purpose_p1,LTV,Status
0,0.0,0.0,0.0,0.009917,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.016385,0.0,1.0,0.010096,0
1,0.0,0.0,0.0,0.009917,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.020533,0.0,0.0,0.008737,0
2,1.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.018044,0.0,1.0,0.011611,0
3,1.0,1.0,0.0,0.006167,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.017422,0.0,1.0,0.008827,0
4,1.0,1.0,0.0,0.085333,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.008711,0.0,0.0,0.009979,0


### Split the data
We now split the data using a 70/30 train/test split.

In [5]:
train_df = data.sample(frac=0.7,random_state=5822)
val_df = data.drop(train_df.index)

In [6]:
train_df.head()

,co-applicant_credit_type_EXP,Gender_Joint,total_units_3U,Upfront_charges,Neg_ammortization_neg_amm,construction_type_mh,Secured_by_land,Security_Type_Indriect,occupancy_type_sr,Region_North,...,age_<25,approv_in_adv_nopre,age_35-44,lump_sum_payment_lpsm,credit_type_CIB,income,loan_limit_ncf,loan_purpose_p1,LTV,Status
78554,1.0,1.0,0.0,0.000000,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.009955,0.0,0.0,0.006438,1
49350,1.0,1.0,0.0,0.039583,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.008089,0.0,0.0,0.011200,0
48569,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.009126,0.0,1.0,0.012346,1
29625,1.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.010059,0.0,0.0,0.012374,0
23503,1.0,1.0,0.0,0.053755,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.008089,0.0,0.0,0.004488,0


In [11]:
val_df.head()

,co-applicant_credit_type_EXP,Gender_Joint,total_units_3U,Upfront_charges,Neg_ammortization_neg_amm,construction_type_mh,Secured_by_land,Security_Type_Indriect,occupancy_type_sr,Region_North,...,age_<25,approv_in_adv_nopre,age_35-44,lump_sum_payment_lpsm,credit_type_CIB,income,loan_limit_ncf,loan_purpose_p1,LTV,Status
3,1.0,1.0,0.0,0.006167,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.017422,0.0,1.0,0.008827,0
8,0.0,0.0,0.0,0.038608,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.006948,0.0,0.0,0.010253,1
10,1.0,1.0,0.0,0.019167,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.006533,0.0,0.0,0.010098,1
14,1.0,1.0,0.0,0.065885,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.009229,0.0,1.0,0.012248,1
16,0.0,0.0,0.0,0.014917,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.009229,0.0,1.0,0.008558,0


# Defining our model
We now define the model for our Bayesian logistic regression as $p(y_i=1|\theta) = \sigma(\beta_0 + \sum\limits_{i=1}^{30} \beta_ix_i)$, where $\theta=\{\beta_i\}_{i=0}^{30}$ and $\sigma$ is the logstic sigmoid. To start, we assume all of our parameters follow a uniform distribution between $[-1,1]$.

In [16]:
formula = "Status ~"
for feat in feat_cols:
    formula += f" {feat} +"
formula = formula[:-2]
formula

'Status ~ co-applicant_credit_type_EXP + Gender_Joint + total_units_3U + Upfront_charges + Neg_ammortization_neg_amm + construction_type_mh + Secured_by_land + Security_Type_Indriect + occupancy_type_sr + Region_North + Interest_rate_spread + Region_North-East + age_45-54 + open_credit_opc + loan_purpose_p4 + rate_of_interest + loan_amount + term + interest_only_int_only + total_units_4U + occupancy_type_ir + age_<25 + approv_in_adv_nopre + age_35-44 + lump_sum_payment_lpsm + credit_type_CIB + income + loan_limit_ncf + loan_purpose_p1 + LTV'

In [19]:
with pm.Model() as blr:
    betas = []
    for i in range(31):
        betas.append(pm.Uniform(f'$\\beta_{{{i}}}$', lower=-1, upper=1))
    logit = betas[0]
    for beta, feat in zip(betas[1:], feat_cols):
        logit += beta * val_df[feat]
    likelihood = pm.invlogit(logit)
    pm.Bernoulli(name='pred', p=likelihood, observed=val_df['Status'])
    

In [26]:
with blr:
    map = pm.find_MAP()